# Alfred, le majordome chargé de trier le courrier : Un exemple de LangGraph

Dans ce *notebook*, **nous allons construire un *workflow* complet pour le traitement des emails en utilisant LangGraph**.

Ce notebook fait parti du cours <a href="https://huggingface.co/learn/agents-course/fr">sur les agents d'Hugging Face</a>, un cours gratuit qui vous guidera, du **niveau débutant à expert**, pour comprendre, utiliser et construire des agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Ce que vous allez apprendre

Dans ce *notebook*, vous apprendrez à :
1. Mettre en place un *workflow* LangGraph
2. Définir l'état et les nœuds pour le traitement des emails
3. Créer un branchement conditionnel dans un graphe
4. Connecter un LLM pour la classification et la génération de contenu
5. Visualiser le graphe du *workflow*
6. Exécuter le *workflow* avec des données d'exemple

In [ ]:
# Installer les paquets nécessaires
%pip install -q langgraph langchain_openai langchain_huggingface

## Configuration de notre environnement

Tout d'abord, importons toutes les bibliothèques nécessaires. LangGraph fournit la structure du graphe, tandis que LangChain offre des interfaces pratiques pour travailler avec les LLM.

In [ ]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Définissez votre clé API OpenAI ici
os.environ["OPENAI_API_KEY"] = "sk-xxxxx"  # Remplacer par votre clé API

# Initialiser notre LLM
model = ChatOpenAI(model="gpt-4o", temperature=0)

## Étape 1 : Définir notre état

Dans LangGraph, **State** est le concept central. Il représente toutes les informations qui circulent dans notre *workflow*.

Pour le système de traitement des emails d'Alfred, nous devons suivre :
- L'email en cours de traitement
- S'il s'agit d'un spam ou non
- Le projet de réponse (pour les courriels légitimes)
- L'historique de la conversation avec le LLM

In [ ]:
class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    spam_reason: Optional[str]
    email_category: Optional[str]
    email_draft: Optional[str]
    messages: List[Dict[str, Any]]

## Étape 2 : Définir nos nœuds

In [ ]:
def read_email(state: EmailState):
    email = state["email"]
    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")
    return {}


def classify_email(state: EmailState):
    email = state["email"]

    prompt = f"""
As Alfred the butler of Mr wayne and it's SECRET identity Batman, analyze this email and determine if it is spam or legitimate and should be brought to Mr wayne's attention.

Email:
From: {email['sender']}
Subject: {email['subject']}
Body: {email['body']}

First, determine if this email is spam.
answer with SPAM or HAM if it's legitimate. Only return the answer
Answer :
    """
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    response_text = response.content.lower()
    print(response_text)
    is_spam = "spam" in response_text and "ham" not in response_text

    if not is_spam:
        new_messages = state.get("messages", []) + [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content}
        ]
    else:
        new_messages = state.get("messages", [])

    return {
        "is_spam": is_spam,
        "messages": new_messages
    }


def handle_spam(state: EmailState):
    print(f"Alfred has marked the email as spam.")
    print("The email has been moved to the spam folder.")
    return {}


def drafting_response(state: EmailState):
    email = state["email"]

    prompt = f"""
As Alfred the butler, draft a polite preliminary response to this email.

Email:
From: {email['sender']}
Subject: {email['subject']}
Body: {email['body']}

Draft a brief, professional response that Mr. Wayne can review and personalize before sending.
    """

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]

    return {
        "email_draft": response.content,
        "messages": new_messages
    }


def notify_mr_wayne(state: EmailState):
    email = state["email"]

    print("\n" + "=" * 50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print("\nI've prepared a draft response for your review:")
    print("-" * 50)
    print(state["email_draft"])
    print("=" * 50 + "\n")

    return {}


# Définir la logique de routage
def route_email(state: EmailState) -> str:
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"


# Créer le graphe
email_graph = StateGraph(EmailState)

# Ajouter des nœuds
email_graph.add_node("read_email", read_email)  # le nœud read_email exécute la fonction read_mail
email_graph.add_node("classify_email", classify_email)  # le nœud classify_email exécutera la fonction classify_email
email_graph.add_node("handle_spam", handle_spam)  # même logique
email_graph.add_node("drafting_response", drafting_response)  # même logique
email_graph.add_node("notify_mr_wayne", notify_mr_wayne)  # même logique


## Étape 3 : Définir notre logique de routage

In [ ]:
# Ajouter des arêtes
email_graph.add_edge(START, "read_email")  # Après le départ, nous accédons au nœud « read_email »

email_graph.add_edge("read_email", "classify_email")  # after_reading nous classifions

# Ajouter des arêtes conditionnelles
email_graph.add_conditional_edges(
    "classify_email",  # après la classification, nous exécutons la fonction « route_email »
    route_email,
    {
        "spam": "handle_spam",  # s'il renvoie « Spam », nous allons au noeud « handle_span »
        "legitimate": "drafting_response"  # et s'il est légitime, nous passons au nœud « drafting_response »
    }
)

# Ajouter les arêtes finales
email_graph.add_edge("handle_spam", END)  # après avoir traité le spam, nous terminons toujours
email_graph.add_edge("drafting_response", "notify_mr_wayne")
email_graph.add_edge("notify_mr_wayne", END)  # après avoir notifié M. Wayne, nous pouvons mettre un terme à l'opération


## Étape 4 : Créer le graphe d'état et définir les arêtes

In [ ]:
# Compiler le graphique
compiled_graph = email_graph.compile()

In [ ]:
from IPython.display import Image, display

display(Image(compiled_graph.get_graph().draw_mermaid_png()))

In [ ]:
 # Exemple de courriels à tester
legitimate_email = {
    "sender": "Joker",
    "subject": "Found you Batman ! ",
    "body": "Mr. Wayne,I found your secret identity ! I know you're batman ! Ther's no denying it, I have proof of that and I'm coming to find you soon. I'll get my revenge. JOKER"
}

spam_email = {
    "sender": "Crypto bro",
    "subject": "The best investment of 2025",
    "body": "Mr Wayne, I just launched an ALT coin and want you to buy some !"
}
# Traiter les emails légitimes
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke({
    "email": legitimate_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

# Traiter les spams
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke({
    "email": spam_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

## Étape 5 : Inspection de notre agent trieur d'emails avec Langfuse 📡

Au fur et à mesure qu'Alfred peaufine l'agent trieur d'emails, il se lasse de déboguer ses exécutions. Les agents, par nature, sont imprévisibles et difficiles à inspecter. Mais comme son objectif est de construire l'ultime agent de détection de spam et de le déployer en production, il a besoin d'une traçabilité solide pour un contrôle et une analyse ultérieurs.

Pour ce faire, Alfred peut utiliser un outil d'observabilité tel que [Langfuse](https://langfuse.com/) pour retracer et surveiller les étapes internes de l'agent.

Tout d'abord, nous devons installer les dépendances nécessaires :

In [ ]:
%pip install -q langfuse

Ensuite, nous définissons les clés de l'API Langfuse et l'adresse de l'hôte en tant que variables d'environnement. Vous pouvez obtenir vos identifiants Langfuse en vous inscrivant à [Langfuse Cloud](https://cloud.langfuse.com) ou à [Langfuse auto-hébergé](https://langfuse.com/self-hosting).

In [ ]:
import os

# Obtenez les clés de votre projet à partir de la page des paramètres du projet : https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 région EU 
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 région US

Nous allons maintenant configurer le [Langfuse `callback_handler`] (https://langfuse.com/docs/integrations/langchain/tracing#add-langfuse-to-your-langchain-application).

In [ ]:
from langfuse.langchain import CallbackHandler

# Initialiser le CallbackHandler Langfuse pour LangGraph/Langchain (traçage)
langfuse_handler = CallbackHandler()

Nous ajoutons ensuite `config={« callbacks » : [langfuse_handler]}` à l'invocation des agents et les exécutons à nouveau.

In [ ]:
# Traiter les emails légitimes
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke(
    input={
        "email": legitimate_email,
        "is_spam": None,
        "draft_response": None,
        "messages": []
    },
    config={"callbacks": [langfuse_handler]}
)

# Traiter les spams
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke(
    input={
        "email": spam_email,
        "is_spam": None,
        "draft_response": None,
        "messages": []
    },
    config={"callbacks": [langfuse_handler]}
)

Alfred est maintenant connecté 🔌 ! Les exécutions de LangGraph sont enregistrées dans Langfuse, ce qui lui donne une visibilité totale sur le comportement de l'agent. Avec cette configuration, il est prêt à revoir les exécutions précédentes et à affiner encore davantage son agent de tri du courrier.

![Example trace in Langfuse](https://langfuse.com/images/cookbook/huggingface-agent-course/langgraph-trace-legit.png)

_[Lien public vers la trace avec l'email légitime](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/f5d6d72e-20af-4357-b232-af44c3728a7b?timestamp=2025-03-17T10%3A13%3A28.413Z&observation=6997ba69-043f-4f77-9445-700a033afba1)_

![Example trace in Langfuse](https://langfuse.com/images/cookbook/huggingface-agent-course/langgraph-trace-spam.png)

_[Lien public vers la trace du spam](https://langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/6e498053-fee4-41fd-b1ab-d534aca15f82?timestamp=2025-03-17T10%3A13%3A30.884Z&observation=84770fc8-4276-4720-914f-bf52738d44ba)_
